## Imprecisões e dúvidas sobre a estrutura cicloviária

Os arquivos fornecidos com as ciclovias, ciclofaixas e ciclorrotas possuem o atributo `extensao_c` que mede o comprimento da via em metros. E o campo `geometry` com a descrição do traçado da via.

Quando calculamos o comprimento desse traçado em `geometry`, gerando o atributo `length`, deveríamos obter algum valor próximo ao `extensao_c`, mas algumas diferenças se destacaram.

In [1]:
import saopaulo.sp_grid as gr

import geopandas as gpd
import json
import pandas as pd
import numpy as np
from ipywidgets import interact_manual, widgets, VBox, HBox, interactive
from IPython.core.display import display, HTML, clear_output
import folium
from folium.plugins import HeatMap
import warnings
import requests
import os
import fnmatch as fnm
warnings.simplefilter('ignore')

from random import randint
from datetime import datetime
import inspect


# SIRGAS 2000 / UTM zone 23S
# http://www.processamentodigital.com.br/2013/07/27/lista-dos-codigos-epsg-mais-utilizados-no-brasil/

protected_lanes = \
        gpd.read_file('../data/sao-paulo/infraestrutura-cicloviaria/cet/Ciclovias.shp')
protected_lanes = protected_lanes.dissolve(by="programa", as_index=False)
protected_lanes.crs = {'init': 'epsg:4326'} 
#protected_lanes.to_crs(epsg='4326', inplace=True)

bike_lanes = \
        gpd.read_file('../data/sao-paulo/infraestrutura-cicloviaria/ciclofaixas.shp')
bike_lanes = bike_lanes.dissolve(by="programa", as_index=False)
#bike_lanes = bike_lanes[bike_lanes['programa'].str.contains("CICLOFAIXA")] # arquivo também contém ciclovia
bike_lanes.crs = {'init': 'epsg:4326'}  
bike_lanes.to_crs(epsg='4326', inplace=True)

sharrow_lanes = \
        gpd.read_file('../data/sao-paulo/infraestrutura-cicloviaria/cet/Ciclorrotas.shp')
sharrow_lanes = sharrow_lanes.dissolve(by="programa", as_index=False)
sharrow_lanes.crs = {'init': 'epsg:4326'}  
#sharrow_lanes.to_crs(epsg='4326', inplace=True)

#assigning variables
protected_color = 'red'  # ciclovias
bike_lane_color = 'green'     # ciclofaixas - antes era trail
sharrow_color = 'orange' # ciclorrotas

style_protected = lambda style:{'color':protected_color, 'weight': 2}
style_lane = lambda style:{'color':bike_lane_color, 'weight': 2}
style_sharrow = lambda style:{'color':sharrow_color, 'weight': 2}
style_zones = lambda x: {'color': 'black', 'weight': 1, 'opacity': 0.3, 'fillOpacity': 0.1}

# map offsets
west_offset=-0.15
east_offset=0.23
north_offset=0.19
south_offset=-0.46

/home/pedro/.local/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
display(protected_lanes.sort_values('inauguracao', ascending=False))
display(bike_lanes.sort_values('inauguraca', ascending=False))

,programa,geometry,inauguracao,extensao_t,extensao_c
43,CICLOFAIXA BARONESA DE MURITIBA,"MULTILINESTRING ((-46.47244 -23.62248, -46.472...",2021-04-24,1405,1428
218,CICLOFAIXA MIRANDA DE AZEVEDO,"MULTILINESTRING ((-46.69314 -23.53395, -46.693...",2021-04-20,68,1534
90,CICLOFAIXA CUSTÓDIO DE SÁ E FARIA,"LINESTRING (-46.50396 -23.61397, -46.50402 -23...",2021-04-19,1122,1122
260,CICLOFAIXA PEDRO BUENO,"MULTILINESTRING ((-46.66239 -23.63770, -46.662...",2021-04-15,321,2842
118,CICLOFAIXA FRANCISCO DE PAULA VICENTE AZEVEDO,"MULTILINESTRING ((-46.75955 -23.53936, -46.759...",2021-04-12,10,1712
...,...,...,...,...,...
254,CICLOFAIXA PASSO DA PÁTRIA,"MULTILINESTRING ((-46.73234 -23.52278, -46.732...",None,161,1765
316,CICLOFAIXA VERBO DIVINO,"LINESTRING (-46.71398 -23.63046, -46.71397 -23...",None,1961,1961
330,CICLOFAIXA WALDEMAR TIETZ,"MULTILINESTRING ((-46.49134 -23.55343, -46.491...",None,1284,1377
365,CICLOVIA OCTALES/ ZAGOTTIS,"MULTILINESTRING ((-46.69965 -23.67552, -46.699...",None,536,1123


,programa,geometry,inauguraca,extensao_c
193,CICLOFAIXA REBOUÇAS,"MULTILINESTRING ((-46.69089 -23.57083, -46.690...",2020/11/26 00:00:00.000,3413
210,CICLOFAIXA SÃO MIGUEL,"MULTILINESTRING ((-46.50223 -23.51158, -46.502...",2020/11/20 00:00:00.000,4926
61,CICLOFAIXA COSTA CARVALHO,"MULTILINESTRING ((-46.70039 -23.56774, -46.700...",2020/10/28 00:00:00.000,1346
25,CICLOFAIXA ASSIS RIBEIRO - TRECHO 5,"MULTILINESTRING ((-46.55045 -23.50972, -46.550...",2020/10/27 00:00:00.000,6051
16,CICLOFAIXA ARICANDUVA,"MULTILINESTRING ((-46.54281 -23.53991, -46.542...",2020/10/23 00:00:00.000,11550
...,...,...,...,...
276,CICLOVIA RIO PINHEIROS,"MULTILINESTRING ((-46.69121 -23.58753, -46.691...",2010/02/27 00:00:00.000,13681
238,CICLOVIA ADUTORA RIO CLARO,"LINESTRING (-46.47626 -23.61152, -46.48001 -23...",2010/01/16 00:00:00.000,7470
268,CICLOVIA PARQUE GUARAPIRANGA,"MULTILINESTRING ((-46.71769 -23.67770, -46.715...",2009/12/01 00:00:00.000,3597
42,CICLOFAIXA CAMINHO VERDE,"MULTILINESTRING ((-46.57556 -23.54042, -46.574...",2008/09/27 00:00:00.000,11582


### Diferenças nas ciclovias

In [3]:
# comparando o comprimento das ciclovias calculado com o fornecidas nos dados
aux_df1 = protected_lanes[['programa', 'extensao_c', 'extensao_t']]
aux_df1['length'] = protected_lanes['geometry'].length * 100000

aux_df1['absolute_diff'] =  abs(aux_df1['length'] - aux_df1['extensao_c'])
aux_df1['relative_diff'] =  abs(aux_df1['length'] - aux_df1['extensao_c'])/aux_df1['extensao_c']

aux_df1['length'] = np.round(aux_df1['length'])
aux_df1['programa'] = aux_df1['programa'].str.slice(stop=35)

display(aux_df1.sort_values('extensao_c', ascending=False))
display(aux_df1.sort_values('absolute_diff', ascending=False))
display(aux_df1.sort_values('relative_diff', ascending=False))

,programa,extensao_c,extensao_t,length,absolute_diff,relative_diff
368,CICLOVIA PARQUE ECOLOGICO DO TIETE,13979,13979,13421.0,557.639596,0.039891
378,CICLOVIA RIO PINHEIROS,13681,6369,19947.0,6265.900472,0.458000
364,CICLOVIA MONOTRILHO VILA PRUDENTE,12694,1298,12201.0,492.840093,0.038825
59,CICLOFAIXA CAMINHO VERDE,11582,1926,11244.0,337.656101,0.029154
27,CICLOFAIXA ARICANDUVA,11550,33,21279.0,9728.998695,0.842338
...,...,...,...,...,...,...
229,CICLOFAIXA NAZARÉ,106,106,189.0,83.201270,0.784918
376,CICLOVIA PORTAO 1 DA USP,74,74,72.0,2.362333,0.031923
79,CICLOFAIXA CONEXÃO FARIA LIMA / R D,55,55,53.0,1.603243,0.029150
184,CICLOFAIXA LIMOEIRO DE ANÁDIA,38,38,36.0,1.811825,0.047680


,programa,extensao_c,extensao_t,length,absolute_diff,relative_diff
27,CICLOFAIXA ARICANDUVA,11550,33,21279.0,9728.998695,0.842338
378,CICLOVIA RIO PINHEIROS,13681,6369,19947.0,6265.900472,0.458000
153,CICLOFAIXA JACU PESSEGO,7405,658,13440.0,6035.420894,0.815047
38,CICLOFAIXA ASSIS RIBEIRO - TRECHO 5,6061,6061,11569.0,5507.571450,0.908690
114,CICLOFAIXA ESTRADA DO IMPERADOR - T,5360,2315,9858.0,4498.066314,0.839191
...,...,...,...,...,...,...
376,CICLOVIA PORTAO 1 DA USP,74,74,72.0,2.362333,0.031923
184,CICLOFAIXA LIMOEIRO DE ANÁDIA,38,38,36.0,1.811825,0.047680
79,CICLOFAIXA CONEXÃO FARIA LIMA / R D,55,55,53.0,1.603243,0.029150
381,CICLOVIA SANTOS DUMONT - TRECHO 1,523,151,522.0,0.582680,0.001114


,programa,extensao_c,extensao_t,length,absolute_diff,relative_diff
34,CICLOFAIXA ASSIS RIBEIRO - TRECHO 1,4,1109,4156.0,4151.966047,1037.991512
113,CICLOFAIXA ESTRADA DO IMPERADOR - T,1170,1170,2285.0,1115.288577,0.953238
286,CICLOFAIXA RUBENS GOMES DE SOUZA,164,164,319.0,154.823845,0.944048
37,CICLOFAIXA ASSIS RIBEIRO - TRECHO 4,722,722,1399.0,677.257889,0.938030
35,CICLOFAIXA ASSIS RIBEIRO - TRECHO 2,1671,1669,3238.0,1566.721326,0.937595
...,...,...,...,...,...,...
327,CICLOFAIXA VINTE E UM DE ABRIL,1287,834,1309.0,21.562575,0.016754
252,CICLOFAIXA PARQUE SÃO LOURENÇO,3393,299,3446.0,52.534461,0.015483
154,CICLOFAIXA JAGUARA,2138,430,2108.0,29.941402,0.014004
381,CICLOVIA SANTOS DUMONT - TRECHO 1,523,151,522.0,0.582680,0.001114


In [8]:
display(aux_df1.iloc[[34, 113, 37, 378]])

aux_map = gr.map_around_sp(the_grid=None,zoom=11,plot_grid=False)

folium.GeoJson(protected_lanes.iloc[34]["geometry"],
                   style_function=style_protected,
                   name='Ciclorrota').add_to(aux_map)
folium.GeoJson(protected_lanes.iloc[113]["geometry"],
                   style_function=style_protected,
                   name='Ciclorrota').add_to(aux_map)
folium.GeoJson(protected_lanes.iloc[37]["geometry"],
                   style_function=style_protected,
                   name='Ciclorrota').add_to(aux_map)
folium.GeoJson(protected_lanes.iloc[378]["geometry"],
                   style_function=style_protected,
                   name='Ciclorrota').add_to(aux_map)


display(aux_map)

,programa,extensao_c,extensao_t,length,absolute_diff,relative_diff
34,CICLOFAIXA ASSIS RIBEIRO - TRECHO 1,4,1109,4156.0,4151.966047,1037.991512
113,CICLOFAIXA ESTRADA DO IMPERADOR - T,1170,1170,2285.0,1115.288577,0.953238
37,CICLOFAIXA ASSIS RIBEIRO - TRECHO 4,722,722,1399.0,677.257889,0.938030
378,CICLOVIA RIO PINHEIROS,13681,6369,19947.0,6265.900472,0.458000


In [7]:
# comparando o comprimento das ciclofaixas calculado com o fornecidas nos dados
aux_df = bike_lanes[['programa', 'extensao_c']]
aux_df['length'] = bike_lanes['geometry'].length * 100000

aux_df['absolute_diff'] =  abs(aux_df['length'] - aux_df['extensao_c'])
aux_df['relative_diff'] =  abs(aux_df['length'] - aux_df['extensao_c'])/aux_df['extensao_c']

aux_df['length'] = np.round(aux_df['length'])
aux_df['programa'] = aux_df['programa'].str.slice(stop=35)

display(aux_df.sort_values('relative_diff', ascending=False))

aux_map = gr.map_around_sp(the_grid=None,zoom=11,plot_grid=False)

#Exemplo Aricanduva 
aux_map = gr.map_around_sp(the_grid=None,zoom=11,plot_grid=False)

folium.GeoJson(bike_lanes.iloc[16]["geometry"],
                   style_function=style_lane,
                   name='Ciclorrota').add_to(aux_map) #Ciclofaixa Aricanduva

#Exemplo Jaguara 
folium.GeoJson(bike_lanes.iloc[108]["geometry"],
                   style_function=style_lane,
                   name='Ciclorrota').add_to(aux_map) #Ciclofaixa Jaguara

#Exemplo Candido 
folium.GeoJson(bike_lanes.iloc[43]["geometry"],
                   style_function=style_lane,
                   name='Ciclorrota').add_to(aux_map) #Ciclofaixa Candido

#Exemplo Calim Eid 
display(aux_df.iloc[[16, 108,43,39,38]])

folium.GeoJson(bike_lanes.iloc[39]["geometry"],
                   style_function=style_protected,
                   name='Ciclorrota').add_to(aux_map) #Ciclofaixa calim eid - trecho 2
folium.GeoJson(bike_lanes.iloc[38]["geometry"],
                   style_function=style_protected,
                   name='Ciclorrota').add_to(aux_map) #Ciclofaixa calim eid - trecho 1
display(aux_map)
# Conclusão: usar o length, calculado, ao invés do comprimento fornecido é mais consistente

,programa,extensao_c,length,absolute_diff,relative_diff
39,CICLOFAIXA CALIM EID - TRECHO 2,1230,7100.0,5870.433927,4.772711
78,CICLOFAIXA ESTRADA DO IMPERADOR - T,1170,2285.0,1115.288577,0.953238
24,CICLOFAIXA ASSIS RIBEIRO - TRECHO 4,722,1399.0,677.257889,0.938030
22,CICLOFAIXA ASSIS RIBEIRO - TRECHO 2,1671,3238.0,1566.721326,0.937595
158,CICLOFAIXA NORMA PIERUCCINI GIANNOT,480,929.0,448.969120,0.935352
...,...,...,...,...,...
117,CICLOFAIXA JARDIM HELENA / SÃO MIGU,8094,8238.0,144.467341,0.017849
228,CICLOFAIXA VINTE E UM DE ABRIL,1287,1309.0,21.562575,0.016754
174,CICLOFAIXA PARQUE SÃO LOURENÇO,3393,3446.0,52.534461,0.015483
108,CICLOFAIXA JAGUARA,2138,2108.0,29.941402,0.014004


,programa,extensao_c,length,absolute_diff,relative_diff
16,CICLOFAIXA ARICANDUVA,11550,21279.0,9728.998695,0.842338
108,CICLOFAIXA JAGUARA,2138,2108.0,29.941402,0.014004
43,CICLOFAIXA CANDIDO MOTTA FILHO,1371,2174.0,803.023847,0.585721
39,CICLOFAIXA CALIM EID - TRECHO 2,1230,7100.0,5870.433927,4.772711
38,CICLOFAIXA CALIM EID - TRECHO 1,1059,1945.0,885.891844,0.836536


In [6]:
# comparando o comprimento das ciclorrotas calculado com o fornecidas nos dados
aux_df2 = sharrow_lanes[['programa', 'extensao_c']]
aux_df2['length'] = sharrow_lanes['geometry'].length * 100000

aux_df2['absolute_diff'] =  abs(aux_df2['length'] - aux_df2['extensao_c'])
aux_df2['relative_diff'] =  abs(aux_df2['length'] - aux_df2['extensao_c'])/aux_df2['extensao_c']

display(aux_df2.sort_values('relative_diff', ascending=False))

aux_map2 = gr.map_around_sp(the_grid=None,zoom=11,plot_grid=False)
for i in range(0,6):
    folium.GeoJson(sharrow_lanes.iloc[i]['geometry'], style_function=style_sharrow, name='Ciclorrota').add_to(aux_map2)

folium.LayerControl().add_to(aux_map2)
display(aux_map2)

,programa,extensao_c,length,absolute_diff,relative_diff
9,CICLORROTA VILA MARIANA,4479,8024.640489,3545.640489,0.791614
5,CICLORROTA JARDINS,4559,7352.381353,2793.381353,0.612718
3,CICLORROTA BROOKLIN,5950,8867.518848,2917.518848,0.490339
4,CICLORROTA JARDIM EUROPA,1288,1550.586429,262.586429,0.203871
2,CICLORROTA BRAZ CUBAS,437,407.104814,29.895186,0.068410
1,CICLORROTA BATURITÉ/DIAMANTE,465,436.514481,28.485519,0.061259
0,CICLORROTA BATURITÉ/ DIAMANTE,83,78.309128,4.690872,0.056517
6,CICLORROTA LAPA,7732,7346.227201,385.772799,0.049893
7,CICLORROTA MOEMA,3205,3111.794885,93.205115,0.029081
8,CICLORROTA MOOCA,3913,3949.446169,36.446169,0.009314
